In [1]:
import os
import json
import cv2
import numpy as np
import pandas as pd


def crop_and_save_polygons(json_folder, image_root, save_folder, ann_output_csv):
    os.makedirs(save_folder, exist_ok=True)
    records = []
    for file in os.listdir(json_folder):
        if not file.endswith(".json"):
            continue
        json_path = os.path.join(json_folder, file)
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        # Tìm tên ảnh đúng (xử lý nếu đường dẫn lạ)
        imgname = os.path.basename(data["imagePath"]).replace("..\\images\\", "")
        image_path = os.path.join(image_root, imgname)
        if not os.path.exists(image_path):
            print("Không tìm thấy ảnh:", image_path)
            continue
        image = cv2.imread(image_path)
        if image is None:
            print("Lỗi đọc ảnh:", image_path)
            continue
        base_name = os.path.splitext(imgname)[0]
        for idx, shape in enumerate(data["shapes"]):
            if shape.get("shape_type", "polygon") != "polygon":
                continue
            label = shape.get("label", "")
            desc = shape.get("description", "")
            points = np.array(shape["points"], dtype=np.float32)
            points_int = np.round(points).astype(np.int32).reshape((-1, 1, 2))
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [points_int], 255)
            x, y, w, h = cv2.boundingRect(points_int)
            crop_img = image[y : y + h, x : x + w].copy()
            crop_mask = mask[y : y + h, x : x + w]
            bg = np.ones_like(crop_img, dtype=np.uint8) * 255  # background trắng
            crop_img_masked = np.where(crop_mask[..., None] == 255, crop_img, bg)
            crop_fname = f"{base_name}_crop_{idx}.jpg"
            out_path = os.path.join(save_folder, crop_fname)
            cv2.imwrite(out_path, crop_img_masked)
            # Lưu annotation cho OCR
            records.append(
                {"filename": crop_fname, "label": label, "description_gt": desc}
            )
    # Lưu bảng annotation
    pd.DataFrame(records).to_csv(ann_output_csv, index=False)
    print("Đã crop xong:", len(records), "vùng, bảng annotation:", ann_output_csv)
    return records


# ==== Chạy thử nghiệm ====
# Thay đổi đúng đường dẫn
json_folder = "./dataset_original/labels"
image_root = "./dataset_original/images"
save_folder = "./dataset_original/crops"
ann_output_csv = os.path.join(save_folder, "crops_gt.csv")

crop_and_save_polygons(json_folder, image_root, save_folder, ann_output_csv)

Đã crop xong: 2284 vùng, bảng annotation: ./dataset_original/crops\crops_gt.csv


[{'filename': '001_heo-cao-boi_F_crop_0.jpg',
  'label': 'brand',
  'description_gt': 'HEO CAO BỒI'},
 {'filename': '001_heo-cao-boi_F_crop_1.jpg',
  'label': 'name',
  'description_gt': 'THỊT VIÊN 3 PHÚT'},
 {'filename': '001_heo-cao-boi_F_crop_2.jpg',
  'label': 'date',
  'description_gt': '150824'},
 {'filename': '001_heo-cao-boi_F_crop_3.jpg',
  'label': 'date',
  'description_gt': '150825'},
 {'filename': '001_heo-cao-boi_F_crop_4.jpg',
  'label': 'weight',
  'description_gt': '200 g'},
 {'filename': '001_heo-cao-boi_F_crop_5.jpg',
  'label': 'weight',
  'description_gt': '100 g'},
 {'filename': '002_mi-ly-modern_F_crop_0.jpg',
  'label': 'brand',
  'description_gt': 'ACECOOK'},
 {'filename': '002_mi-ly-modern_F_crop_1.jpg',
  'label': 'name',
  'description_gt': 'MÌ LY MODERN'},
 {'filename': '003_banh-crepe-sua-tuoi_B_crop_0.jpg',
  'label': 'brand',
  'description_gt': 'SweetHome Bakery'},
 {'filename': '003_banh-crepe-sua-tuoi_B_crop_1.jpg',
  'label': 'name',
  'description_g